In [269]:
#vocab_sign = ['+', '-', 'low', 'high', 'pos']
vocab_sign_pos = [
    '+', 
    'pos', 'positive',
    'high', 'hi', 
    'bright'
]
vocab_sign_neg = [
    '-', '–', '—', '−', # hyphen, en dash, em dash, minus
    'neg', 'negative', 
    'lo', 'low',
    'dim'
]
vocab_pr = [
    'CD4', 'CD45RA', 'CD45', 'CD45RO', 'CD62L', 'CCR7', 'CD127',
    'CD27', 'CD28', 'CD122', 'CD8a', 'CD8', 'CD3', 'Thy1', '4-1BB', 
    'CCR7', 'RORgt', 'CD95', 'CD122'
]
vocab = vocab_sign_pos + vocab_sign_neg + vocab_pr
CASES = [
    ('CD4+CD45RA+CD45RO-4-1BB-CD62L+++CCR7loCD127posCD27positiveCD28hiCD95+CD122+', [
        'CD4+', 'CD45RA+', 'CD45RO-', '4-1BB-', 'CD62L+', 'CCR7', 'CD127+', 'CD28+.', 'CD95+.', 'CD122+.'
    ], ['CD95', 'CD28', 'CD122']),
    #('CD4+CD45RO/RBbright', ['CD4+', 'CD45RO/RB+.']),
#     ('CD3CD4PBMC', ['CD3', 'CD4', 'PBMC'])
#     'CD95CD4negativeCD45RApositiveCD45RO-CD62Lpos',
#     'CD3CD4CD8alow',
#     'CD3CD4PBMC',
#     'CD3CD4CD99+IL-12',
#     'Thy1.1+OT-1+CD8+',
#     'CD8-4-1BB-CCR7highCD45RO+',
#     'CD4-CD8+CD3+RORγt+T',
#     'CD4+Foxp3gfp+',
#     'CD4+CD45RO/RBbright',
#     'CD4+CD25+CD45RO+IL-7Ralphalow',
#     'CD4+CD8-Vbeta3hi'
]

In [270]:
import heapq

def match(string, words, max_unmatched_len):
    """Recursively break string input using known vocabulary (with unknown substrings of maximum size)
    
    Generates tuples with values so far for (in this order):
        - count of unmatched characters
        - count of unmatched words 
        - count of matched words
        - list of (substring, is_known) tuples where substring is a word to be broken out and is_known
            is a boolean indicator of whether or not that word was present in the provided word list
    """
    if len(string) == 0:
        yield 0, 0, 0, []
        return
    
    # Loop through oov offsets up to at most `max_unmatched_len`
    n = min(max_unmatched_len + 1, len(string))
    matched = False
    for u in range(n):
        # Ignore u characters at beginning of current candidate and run
        # prefix search for all substrings, taking the largest one possible
        substr = string[u:]
        for i in range(1, len(substr)+1):
            prefix = substr[:i]
            tail = None if i >= len(substr) else substr[:(i+1)]
            if prefix not in words or (tail is not None and tail in words):
                continue
            # Recursion for substring starting at index u + i
            for ctcu, ctwu, ctwm, tokens in match(substr[i:], words, max_unmatched_len):
                matched = True
                tknu, tknm = [] if u == 0 else [(string[:u], False)], [(prefix, True)]
                yield ctcu + u, ctwu + len(tknu), ctwm + len(tknm), tknu + tknm + tokens 
        # Break when any oov offset results in a prefix hit 
        # (this minimizes the length of oov substrings)
        if matched:
            break
    # Return trailing unmatched substring, if necessary
    if not matched:
        yield len(string), 1, 0, [(string, False)]
            

def split(string, words, max_unmatched_len=0, n_results=1, mode='most_words'):
    """Partition a string into words using a controlled vocabulary
    
    This is a solution to the Word Break Problem that includes:
        - Unknown words of at most length `max_unmatched_len`
        - Greedy maximum length known word partitioning
        - Greedy minimum length unknown word partitioning
        
    Example:
    
        words = ["this", "is", "the", "famous", "Word", "break", "b", "r", "e", "a", "k", "br", "problem"]
        string = "WordbreakINGproblem"
        results = split(string, words, max_unmatched_len=3, mode='most_words', n_results=1)[0]
        print([w if known else '|' + w + '|' for w, known in results[-1]])
        >>> ['Word', 'br', 'e', 'a', 'k', '|ING|', 'problem']
        
        results = split(string, words, max_unmatched_len=3, mode='least_words', n_results=1)[0]
        print([w if known else '|' + w + '|' for w, known in results[-1]])
        >>> ['Word', 'break', '|ING|', 'problem']
        
    Returns:
        A list of tuples with values (in this order):
            - count of unmatched characters
            - count of unmatched words 
            - count of matched words
            - list of (substring, is_known) tuples where substring is a word to be broken out and is_known
                is a boolean indicator of whether or not that word was present in the provided word list
    """
    if mode not in ['most_words', 'least_words']:
        raise ValueError(f'Mode must be one of "most_words" or "least_words" not "{mode}"')
    # Limit max unmatched length to length of input string (default to 0 if None)
    max_unmatched_len = min(max(max_unmatched_len or 0, 0), len(string))
    mode = (-1, -1) if mode == 'least_words' else (-1, 1)
    res = []

    # Get unmatched char count, unmatched word count, matched word count, and token list (in that order)
    for ctcu, ctwu, ctwm, tokens in match(string, words, max_unmatched_len):
        ctwt = ctwu + ctwm
        # Minimize number of unmatched characters, maximize or minimize 
        # number of total tokens based on mode
        key = (ctcu * mode[0], ctwt * mode[1])
        item = (key, ctcu, ctwu, ctwm, ctwt, tokens)
        # Push onto bounded heap
        (heapq.heappush if len(res) < n_results else heapq.heappushpop)(res, item)

    # Return all requested results except for those used in sorting heap entry
    return [r[1:] for r in heapq.nlargest(n_results, res)]

for r in split(CASES[0][0], vocab, 8, n_results=2):
    print(r)

(0, 0, 24, 24, [('CD4', True), ('+', True), ('CD45RA', True), ('+', True), ('CD45RO', True), ('-', True), ('4-1BB', True), ('-', True), ('CD62L', True), ('+', True), ('+', True), ('+', True), ('CCR7', True), ('lo', True), ('CD127', True), ('pos', True), ('CD27', True), ('positive', True), ('CD28', True), ('hi', True), ('CD95', True), ('+', True), ('CD122', True), ('+', True)])
(2, 1, 24, 25, [('CD4', True), ('+', True), ('CD45RA', True), ('+', True), ('CD45', True), ('RO', False), ('-', True), ('4-1BB', True), ('-', True), ('CD62L', True), ('+', True), ('+', True), ('+', True), ('CCR7', True), ('lo', True), ('CD127', True), ('pos', True), ('CD27', True), ('positive', True), ('CD28', True), ('hi', True), ('CD95', True), ('+', True), ('CD122', True), ('+', True)])


In [271]:
# results = split(CASES[0][0], vocab, 8, n_results=1)[0][-1]
# groups = [(k, list(g)) for k, g in itertools.groupby(results, lambda v: v[0] in vocab_sign_pos + vocab_sign_neg)]
# for (k, g), (kn, gn) in list(zip(groups, groups[1:] + [(None, None)])):
#     # If first token group is for signs or on the last token group, yield
#     if k or kn is None:
#         print(g)
#     print(k, kn, gn)

In [272]:
import itertools

class ProteinToken(object):
    
    def __init__(self, token_text, sign_text, sign_value):
        self.token_text = token_text
        self.sign_text = sign_text
        self.sign_value = sign_value
        self.text = (token_text or '') + (sign_text or '')
        
    @property
    def sign_value_text(self):
        sign = ''
        if self.sign_value == 1:
            sign = '⁺'
        elif self.sign_value == -1:
            sign = '⁻'
        return sign
    
    def to_string(self, show_sign_text=True):
        return ''.join([
            self.token_text or '', 
            self.sign_value_text or '', 
            '(' + self.sign_text + ')' if self.sign_text and show_sign_text else ''
        ])
    
    def __repr__(self):
        return self.to_string(show_sign_text=True)

        
class ProteinTokenizer(object):
    
    def __init__(self, vocab_pr, vocab_sign_pos, vocab_sign_neg):
        self.vocab = vocab_pr + vocab_sign_pos + vocab_sign_neg
        self.vocab_sign = {**{v:1 for v in vocab_sign_pos}, **{v:-1 for v in vocab_sign_neg}}
        
    def tokenize(self, string, max_unmatched_len=8):
        splits = split(string, self.vocab, max_unmatched_len=max_unmatched_len, n_results=1)
        if not splits:
            yield ProteinToken(string, None, 0)
            return
        
        # Get last item in first result (i.e. list of tokens)
        tokens = splits[0][-1]

        # Group tokens into consecutive sign or non-sign lists
        # (group by here will create a group every time that changes, not in aggregate by True/False)
        groups = [(k, list(g)) for k, g in itertools.groupby(tokens, lambda v: v[0] in self.vocab_sign)]

        # Yield collapsed tokens
        none = [(None, None)]
        for (kp, gp), (k, g), (kn, gn) in zip(none + groups[:-1], groups, groups[1:] + none):
            sn_txt, pr_txt, sn_val = None, None, 0

            # If on a non-sign group, pull in next group (for a sign) 
            if not k:
                pr_txt = ''.join([v[0] for v in g])
                sn_txt = None if gn is None else [v[0] for v in gn]
            # Otherwise if the first group is a sign group
            elif kp is None:
                sn_txt = [v[0] for v in g]

            # Resolve sign string to integer value
            if sn_txt:
                signs = set([self.vocab_sign.get(v, 0) for v in sn_txt]) - set([0])
                # Only set sign if no conflicts exist
                if len(signs) == 1:
                    sn_val = list(signs)[0]
                sn_txt = ''.join(sn_txt)
                
            # Yield if on a collapsed token group
            if sn_txt or pr_txt:
                yield ProteinToken(pr_txt, sn_txt, sn_val)

In [273]:
def test_cases():
    for c in CASES:
        v = list(set(vocab) - set(c[2] if len(c) > 2 else []))
        tokenizer = ProteinTokenizer(v, vocab_sign_pos, vocab_sign_neg)
        #tokens = split(c[0], v, max_unmatched_len=8, n_results=1)
        tokens = tokenizer.tokenize(c[0])
        print(list(tokens))
        
test_cases()

[CD4⁺(+), CD45RA⁺(+), CD45RO⁻(-), 4-1BB⁻(-), CD62L⁺(+++), CCR7⁻(lo), CD127⁺(pos), CD27⁺(positive), CD28⁺(hi), CD95⁺(+), CD122⁺(+)]
